# O problema

Gerar relatórios iniciais a serem apresentados em uma reunião de investidores e acionistas, explicando a quantidade de vinhos exportados e os fatores externos que podem vir a surgir e que interferem nas análises:

1. Dados climáticos.
2. Dados demográficos.
3. Dados econômicos.
4. Dados de avaliações de vinhos.

O head de dados pediram para que você construísse uma tabela contendo as seguintes informações:

1. País de origem (Brasil).
2. País de destino.
3. Quantidade em litros de vinho exportado (utilize: 1kg = 1l).
4. Valor em US$.

Seu objetivo é dizer o montante de venda de exportação nos últimos 15 anos, separando a análise por país e trazendo quais as propecções futuras e possíveis ações para uma melhoria nas exportações. Construa gráficos atraentes e que passem a ideia central para que os acionistas e investidores possam seguir em frente com suas ações.

fontes: 

http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_06

https://pt.wikipedia.org/wiki/Lista_de_pa%C3%ADses_por_popula%C3%A7%C3%A3o

## Importando bibliotecas

In [209]:
import pandas as pd

import warnings 
warnings.filterwarnings(action = 'ignore')

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import plotly.express as px

## Importando dados

In [210]:
dados = pd.read_csv('../dados_uteis/dados_uteis.csv')
dados.head()

## Análise exploratória

### Valores negociados

In [248]:
evolucao_exportacao = dados.groupby('ano').sum().drop('pais', axis = 1).reset_index()
fig_evolucao_valores_exportacao = px.line(
    data_frame=evolucao_exportacao, 
    x='ano', 
    y='valor_exportacao',
    color_discrete_sequence=['#9B3D83'],
    labels={
        'ano': 'Ano',
        'valor_exportacao': 'Valor US$'
    }
)
fig_evolucao_valores_exportacao.update_layout(
    title='', 
    xaxis_title='Período', 
    yaxis_title='Valor (US$)', 
    showlegend=False, 
    width=600,
    height=400
)

In [249]:
acumulado_top_10_mercados = pd.DataFrame(dados.groupby('pais').sum()[['quantidade_exportacao', 'valor_exportacao']])
selecao = acumulado_top_10_mercados.sort_values('valor_exportacao', ascending = False).head(10).index #selecionando os 10 maiores mercados de exportação
acumulado_top_10_mercados = acumulado_top_10_mercados.loc[selecao]
acumulado_top_10_mercados.reset_index(inplace = True)
fig_acumulado_top_10_mercados_bar = px.bar(
    data_frame = acumulado_top_10_mercados,
    x = 'pais',
    y = 'valor_exportacao',
    color='valor_exportacao',
    color_continuous_scale=px.colors.sequential.Magenta,
    #color_discrete_sequence = ['#673E69'],
    labels={
        'pais': 'País',
        'valor_exportacao': 'Valor US$'
    }
)
fig_acumulado_top_10_mercados_bar.update_layout(
    title='',  
    showlegend=False, 
    bargap=0.1,
    width=600,
    height=400
)

In [250]:
fig_acumulado_top_10_mercados_bar = px.treemap(
    data_frame = acumulado_top_10_mercados,
    path = ['pais'], 
    values = 'valor_exportacao',
    color='valor_exportacao',
    color_continuous_scale=px.colors.sequential.Magenta,
    #color_discrete_sequence = ['#673E69'],
    labels={
        'pais': 'País',
        'valor_exportacao': 'Valor US$'
    }
)
fig_acumulado_top_10_mercados_bar.update_layout(
    title='',  
    showlegend=False, 
    width=600,
    height=400
)

In [251]:
selecao = acumulado_top_10_mercados['pais'].head().to_list()
evolucao_top_5 = dados[dados.pais.isin(selecao)]
fig_evolucao_valores_top_5_line = px.line(
    data_frame = evolucao_top_5, 
    x = 'ano', 
    y = 'valor_exportacao',
    color = 'pais',
    color_discrete_sequence=px.colors.sequential.Magenta_r,
    labels={
        'pais': 'País',
        'valor_exportacao': 'Valor US$',
        'ano': 'Ano'
    }
)
fig_evolucao_valores_top_5_line.update_layout(
    title = '', 
    legend_title = 'Legenda',
    width=600,
    height=400
)

### Quantidades negociadas

In [268]:
fig_evolucao_valores_exportacao = px.line(
    data_frame=evolucao_exportacao, 
    x='ano', 
    y='quantidade_exportacao',
    color_discrete_sequence=['#3EA85A'],
    labels={
        'pais': 'País',
        'quantidade_exportacao': 'Quantidade (Litro)',
        'ano': 'Ano'
    }
)
fig_evolucao_valores_exportacao.update_layout(
    title='', 
    xaxis_title='Período', 
    yaxis_title='Valor (US$)', 
    showlegend=False, 
    width=600,
    height=400
)

In [253]:
fig_acumulado_top_10_mercados_bar = px.bar(
    data_frame = acumulado_top_10_mercados,
    x = 'pais',
    y = 'quantidade_exportacao',
    color='quantidade_exportacao',
    color_continuous_scale=px.colors.sequential.Greens,
    labels={
        'pais': 'País',
        'quantidade_exportacao': 'Quantidade (Litro)'
    }
)
fig_acumulado_top_10_mercados_bar.update_layout(
    title='',  
    showlegend=False, 
    bargap=0.1,
    width=600,
    height=400
)

In [254]:
fig_acumulado_top_10_mercados_bar = px.treemap(
    data_frame = acumulado_top_10_mercados,
    path = ['pais'], 
    values = 'quantidade_exportacao',
    color='quantidade_exportacao',
    color_continuous_scale=px.colors.sequential.Greens,
    labels={
        'pais': 'País',
        'quantidade_exportacao': 'Quantidade (Litro)'
    }
)
fig_acumulado_top_10_mercados_bar.update_layout(
    title='',  
    showlegend=False, 
    width=600,
    height=400
)

In [255]:
fig_evolucao_valores_top_5_line = px.line(
    data_frame = evolucao_top_5, 
    x = 'ano', 
    y = 'quantidade_exportacao',
    color = 'pais',
    color_discrete_sequence=px.colors.sequential.Greens,
    labels={
        'pais': 'País',
        'quantidade_exportacao': 'Quantidade (Litro)',
        'ano': 'Ano'
    }
)
fig_evolucao_valores_top_5_line.update_layout(
    title = '', 
    legend_title = 'Legenda',
    width=600,
    height=400
)

### Outras estatísticas

In [256]:
fig_evolucao_valores_quantidade_exportacao = px.line(
    data_frame=evolucao_exportacao, 
    x='ano', 
    y=['valor_exportacao', 'quantidade_exportacao'],
    labels={
        'valor_exportacao': 'quantidade_exportacao',
        'quantidade_exportacao': 'Quantidade (Litro)',
        'ano': 'Ano'
    },
    color_discrete_map = {
        'quantidade_exportacao': '#9B3D83',
        'valor_exportacao': '#3EA85A'
    }
)
nome_variaveis = {'quantidade_exportacao': 'Quantidade (Litros)', 'valor_exportacao': 'Valor (US$)'}
fig_evolucao_valores_quantidade_exportacao.for_each_trace(lambda x: x.update(
        name = nome_variaveis[x.name],
        legendgroup = nome_variaveis[x.name],
        hovertemplate = x.hovertemplate.replace(x.name, nome_variaveis[x.name])
    )
)
fig_evolucao_valores_quantidade_exportacao.update_layout(
    title = '', 
    yaxis_title = 'Valores', 
    legend_title = 'Legenda', 
    width = 600, 
    height = 400
    )

In [257]:
evolucao_preco_medio_por_litro = dados.groupby('ano').mean('valor_exportacao_por_litro').round(2).reset_index()
evolucao_preco_medio_por_litro.head()
fig_evolucao_preco_medio_por_litro = px.line(
        data_frame=evolucao_preco_medio_por_litro, 
        x='ano', 
        y='valor_exportacao_por_litro',
        color_discrete_sequence = ['#9B3D83'],
        labels={
        'valor_exportacao_por_litro': 'Valor (US$)',
        'quantidade_exportacao': 'Quantidade (Litro)',
        'ano': 'Ano'
        }
)
fig_evolucao_preco_medio_por_litro.update_layout(
        title = '', 
        width=600,
        height=400
)


In [258]:
selecao = acumulado_top_10_mercados['pais'].to_list()
evolucao_top_10 = dados[dados.pais.isin(selecao)]
preco_medio_por_litro_top_10 = evolucao_top_10[['pais', 'valor_exportacao_por_litro']].groupby('pais').mean().round(2).reset_index()
fig_preco_medio_por_litro_top_10_bar = px.bar(
    data_frame=preco_medio_por_litro_top_10, 
    x='pais', 
    y='valor_exportacao_por_litro',
    color='valor_exportacao_por_litro',
    color_continuous_scale=px.colors.sequential.Magenta,
    labels={
        'pais': 'País',
        'valor_exportacao_por_litro': 'Valor (US$)'
    }
)
fig_preco_medio_por_litro_top_10_bar.update_layout(
    title = '', 
    xaxis_title = 'Países', 
    yaxis_title = 'Valor (US$)', 
    width=600,
    height=400,
    bargap=0.1
)

In [259]:
fig_preco_medio_por_litro_top_10_treemap = px.treemap(
    data_frame=preco_medio_por_litro_top_10, 
    path =['pais'], 
    values = 'valor_exportacao_por_litro', 
    color='valor_exportacao_por_litro',
    color_continuous_scale=px.colors.sequential.Magenta,
    labels={
        'pais': 'País',
        'valor_exportacao_por_litro': 'Valor (US$)'
    }
)
fig_preco_medio_por_litro_top_10_treemap.update_layout(
    title='',
    showlegend=False,
    width=600,  
    height=400
)

In [260]:
fig_evolucao_valor_por_litro_top_5 = px.line(
    data_frame = evolucao_top_5, 
    x = 'ano', 
    y = 'valor_exportacao_por_litro',
    color = 'pais',
    color_discrete_sequence=px.colors.sequential.Magenta,
    labels={
        'ano': 'Ano',
        'pais': 'País',
        'valor_exportacao_por_litro': 'Valor (US$)'
    }
)
fig_evolucao_valor_por_litro_top_5.update_layout(
    title = '',  
    legend_title = 'Legenda',
    width=600,
    height=400
)

In [261]:
percentual_exportacao = dados[['ano', 'pais', 'percentual_exportacao']].round(2)
evolucao_percentual_medio_exportacao = percentual_exportacao.groupby('ano').mean('percentual_exportacao').round(2).reset_index()
evolucao_percentual_medio_exportacao.head()
fig_evolucao_percentual_medio_exportacao = px.line(
    data_frame=evolucao_percentual_medio_exportacao, 
    x='ano', 
    y='percentual_exportacao',
    color_discrete_sequence = ['#9B3D83'],
    labels={
        'ano': 'Ano',
        'percentual_exportacao': 'Percentual médio'
    }
)
fig_evolucao_percentual_medio_exportacao.update_layout(
    title = '', 
    xaxis_title = 'Período', 
    yaxis_title = '%',
    width=600,
    height=400
)

In [262]:
percentual_medio_exportacao_top_10 = evolucao_top_10.groupby('pais').mean('percentual_exportacao').round(2).reset_index()
percentual_medio_exportacao_top_10.head()
fig_percentual_medio_exportacao_top_10_pie = px.pie(
    data_frame=percentual_medio_exportacao_top_10, 
    values='percentual_exportacao', 
    names='pais',
    color='percentual_exportacao',
    color_discrete_sequence=px.colors.sequential.Magenta,
    labels={
        'pais': 'País',
        'percentual_exportacao': 'Percentual médio'
    }
)
fig_percentual_medio_exportacao_top_10_pie.update_layout(
    title = '', 
    width=600,
    height=400,
)

In [263]:
fig_percentual_medio_exportacao_top_10_treemap = px.treemap(
    data_frame=percentual_medio_exportacao_top_10, 
    path=['pais'], 
    values='percentual_exportacao',
    color='percentual_exportacao',
    color_continuous_scale=px.colors.sequential.Magenta,
    labels={
        'pais': 'País',
        'percentual_exportacao': 'Percentual médio'
    }
)
fig_percentual_medio_exportacao_top_10_treemap.update_layout(
    title = '', 
    width=600,
    height=400,
)

In [264]:
evolucao_percentual_medio_exportacao_top_5 = evolucao_top_5.groupby(['ano', 'pais']).mean('percentual_exportacao').round(2).reset_index()
evolucao_percentual_medio_exportacao_top_5.head()
fig_evolucao_percentual_medio_exportacao_top_5 = px.line(
    data_frame = evolucao_percentual_medio_exportacao_top_5, 
    x = 'ano', 
    y = 'percentual_exportacao',
    color = 'pais',
    color_discrete_sequence=px.colors.sequential.Magenta_r,
    labels={
        'ano': 'Ano',
        'pais': 'País',
        'percentual_exportacao': 'Percentual médio'
    }
)
fig_evolucao_percentual_medio_exportacao_top_5.update_layout(
    title = 'Time series', 
    xaxis_title = 'Período', 
    yaxis_title = '%', 
    legend_title = 'Legenda',
    width=600,
    height = 400)

In [266]:
index = evolucao_top_10.query('quantidade_exportacao == 0').index.to_list()
distribuicao_quantidade_valor_top_10 = evolucao_top_10.drop(index)
distribuicao_quantidade_valor_top_10.head()
fig_distribuicao_valor_top_10 = px.scatter(
    data_frame=distribuicao_quantidade_valor_top_10, 
    x='pais', 
    y='ano', 
    size='valor_exportacao', 
    color='valor_exportacao',
    color_continuous_scale=px.colors.sequential.Magenta,
    labels={
        'ano': 'Ano',
        'pais': 'País',
        'valor_exportacao': 'Valor US$'
    }
)
fig_distribuicao_valor_top_10.update_layout(
    title='',
    width=600,
    height=400
)
fig_distribuicao_valor_top_10.update_traces(marker=dict(size=3.5 * distribuicao_quantidade_valor_top_10['valor_exportacao']))

In [267]:
fig_distribuicao_quantidade_top_10=px.scatter(
    data_frame=distribuicao_quantidade_valor_top_10, 
    x='pais', 
    y='ano', 
    size='quantidade_exportacao', 
    color='quantidade_exportacao',
    color_continuous_scale=px.colors.sequential.Greens,
    labels={
        'ano': 'Ano',
        'pais': 'País',
        'quantidade_exportacao': 'Quantidade (Litros)'
    }
)
fig_distribuicao_quantidade_top_10.update_layout(
    title='', 
    width=600,
    height=400
)
fig_distribuicao_quantidade_top_10.update_traces(marker=dict(size=3.5 * distribuicao_quantidade_valor_top_10['quantidade_exportacao']))